In [1]:
import sys
#!{sys.executable} -m pip install efficientnet
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
# Get pretrained model 
from tensorflow.keras.applications.efficientnet import EfficientNetB0,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image

# Get Connection
import boto3
from boto3 import Session

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Handle Images
from PIL import Image
from io import BytesIO
import io

session = Session()
credentials = session.get_credentials()

#aws access credentials
current_credentials = credentials.get_frozen_credentials()

client = boto3.client(
    's3',
    aws_access_key_id=current_credentials.access_key,
    aws_secret_access_key=current_credentials.secret_key,
    aws_session_token=current_credentials.token)

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [3]:
bucket_name = 'tellisa-collage-gan'
#img_inst = client.get_object(Bucket=bucket_name,Key='collage_gan/train/_.lunatico.__pic#0.jpg')

In [4]:
def get_s3_keys(bucket):
    """Get a list of keys in an S3 bucket."""
    keys = []
    resp = client.list_objects_v2(Bucket=bucket)
    for obj in resp['Contents']:
        keys.append(obj['Key'])
    return keys

In [5]:
collageKeys = np.array(get_s3_keys(bucket_name))
collageKeys = collageKeys[14:]

In [6]:
import concurrent.futures 



def get_image(img_key):
    bucket_name = 'tellisa-collage-gan'
    img_inst = client.get_object(Bucket=bucket_name,Key=img_key)
    #img_inst = Image.open(io.BytesIO(img_inst['Body'].read()))
    #print('step 1')
    #model = EfficientNetB0(include_top = False, weights = 'imagenet', pooling='avg')
    #x = image.img_to_array(img_inst)
    #x = np.expand_dims(x,axis=0)
    #x = preprocess_input(x)
    #print(x)
    #preds = model.predict(x)
    #curr_df = pd.DataFrame(preds[0]).T
    return img_inst



In [7]:
imageMetadata = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(get_image,collageKeys)
    for result in results:
        imageMetadata.append(result)

In [8]:
imageMetadata[0]

{'ResponseMetadata': {'RequestId': 'ZKFVRG4YPB92CDNB',
  'HostId': 'CM5e172oahNBHRVxcwQtEmMPnLbQdLJ5GX1UWjtQt7dKp3/GNCo8COSR/HdAFMSK0OHGUzaX1mU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'CM5e172oahNBHRVxcwQtEmMPnLbQdLJ5GX1UWjtQt7dKp3/GNCo8COSR/HdAFMSK0OHGUzaX1mU=',
   'x-amz-request-id': 'ZKFVRG4YPB92CDNB',
   'date': 'Sat, 21 Jan 2023 17:37:42 GMT',
   'last-modified': 'Wed, 23 Nov 2022 00:57:03 GMT',
   'etag': '"77fb71617b0a87bb0135ee115e421335"',
   'x-amz-server-side-encryption': 'AES256',
   'accept-ranges': 'bytes',
   'content-type': 'image/jpeg',
   'server': 'AmazonS3',
   'content-length': '50221'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2022, 11, 23, 0, 57, 3, tzinfo=tzutc()),
 'ContentLength': 50221,
 'ETag': '"77fb71617b0a87bb0135ee115e421335"',
 'ContentType': 'image/jpeg',
 'ServerSideEncryption': 'AES256',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f1f735d9ac8>}